# Podstawy Przetwarzania Dużych Zbiorów Danych

Grupa IBD4 Poniedziałki 8:30

Łukasz Moskwa 247739 

Artur Szewczykowski 252546



# Część 1
 
1. https://github.com/GoogleCloudPlatform/covid-19-open-data/blob/main/docs/table-by-sex.md
2. https://medium.com/artificial-intelligence-101/data-analysis-with-google-bigquery-exploring-the-covid-19-open-data-public-dataset-2d246c66422d

# Część 2

In [27]:
# 2.5 Uruchom klienta BigQuery z poziomu Pythona.

import os

import pandas as pd
from google.cloud import bigquery

In [28]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=".env/propane-flag-452609-u3-1feb9eb63dc4.json" # lokalizacja pobranego klucza z punktu 1.4.
client = bigquery.Client() 


In [29]:
# 2.6 Pobierz dane do obiektu DataFrame.

query = ('select * from bigquery-public-data.covid19_open_data.covid19_open_data limit 10')
query_job = client.query(query)    
query_result = query_job.result()  
df = query_result.to_dataframe()
df


,location_key,date,place_id,wikidata_id,datacommons_id,country_code,country_name,iso_3166_1_alpha_2,iso_3166_1_alpha_3,aggregation_level,...,cumulative_vaccine_doses_administered_pfizer,new_persons_fully_vaccinated_moderna,cumulative_persons_fully_vaccinated_moderna,new_vaccine_doses_administered_moderna,cumulative_vaccine_doses_administered_moderna,new_persons_fully_vaccinated_janssen,cumulative_persons_fully_vaccinated_janssen,new_vaccine_doses_administered_janssen,cumulative_vaccine_doses_administered_janssen,location_geometry
0,ES_CT,2022-05-14,ChIJ8_UwhdxbpBIRUMijIeD6AAE,Q5705,nuts/ES51,ES,Spain,ES,ESP,1,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(1.537778 41.8375)
1,ES_CT,2021-03-28,ChIJ8_UwhdxbpBIRUMijIeD6AAE,Q5705,nuts/ES51,ES,Spain,ES,ESP,1,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(1.537778 41.8375)
2,ES_CT,2021-03-21,ChIJ8_UwhdxbpBIRUMijIeD6AAE,Q5705,nuts/ES51,ES,Spain,ES,ESP,1,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(1.537778 41.8375)
3,ES_CT,2021-05-28,ChIJ8_UwhdxbpBIRUMijIeD6AAE,Q5705,nuts/ES51,ES,Spain,ES,ESP,1,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(1.537778 41.8375)
4,ES_CT,2022-02-11,ChIJ8_UwhdxbpBIRUMijIeD6AAE,Q5705,nuts/ES51,ES,Spain,ES,ESP,1,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(1.537778 41.8375)
5,ES_CT,2021-01-04,ChIJ8_UwhdxbpBIRUMijIeD6AAE,Q5705,nuts/ES51,ES,Spain,ES,ESP,1,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(1.537778 41.8375)
6,ES_CT,2020-08-10,ChIJ8_UwhdxbpBIRUMijIeD6AAE,Q5705,nuts/ES51,ES,Spain,ES,ESP,1,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(1.537778 41.8375)
7,US_AL,2020-12-05,ChIJdf5LHzR_hogR6czIUzU0VV4,Q173,geoId/01,US,United States of America,US,USA,1,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-86.7 32.7)
8,US_AL,2020-11-13,ChIJdf5LHzR_hogR6czIUzU0VV4,Q173,geoId/01,US,United States of America,US,USA,1,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-86.7 32.7)
9,US_AL,2020-08-05,ChIJdf5LHzR_hogR6czIUzU0VV4,Q173,geoId/01,US,United States of America,US,USA,1,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-86.7 32.7)


# Część 3
Misja dodatkowa - Poznanie danych

### 3.1 Ile jest zapisanych wierszy z danymi.

In [30]:
dataset_ref = client.dataset('covid19_open_data', project='bigquery-public-data')
table_ref = dataset_ref.table('covid19_open_data')
table = client.get_table(table_ref)

rows = table.num_rows

rows

22756333

### 3.2 Ile krajów jest uwzględnionych w danych.

In [31]:
query = ("""
SELECT COUNT(DISTINCT country_code) AS num_countries
FROM bigquery-public-data.covid19_open_data.covid19_open_data

""")

query_job = client.query(query)
result = query_job.result()

number_of_country = result.to_dataframe()

# Zwraca 246, ponieważ niektóre tereny podległe takie jak "French Southern Territories", "Guam" itd.
# Co jest niby okey ale nie są to państwa bo ich jest 195
number_of_country.iloc[0, 0]


246

 ### 3.3. W jaki sposób zapisywane są dzienne informacje dla krajów.

In [32]:
# Dodają najczęsciej raz dziennie ale zdarza się, że kraj ma 2 wpisy ponieważ robi to dla innego aggregation_level
query = ("""
SELECT *
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE  aggregation_level = 0 AND country_name = 'Poland'
ORDER BY date ASC 
""")

country_all = client.query(query).to_dataframe()
country_all['date']

0      2020-01-01
1      2020-01-02
2      2020-01-03
3      2020-01-04
4      2020-01-05
          ...    
986    2022-09-13
987    2022-09-14
988    2022-09-15
989    2022-09-16
990    2022-09-17
Name: date, Length: 991, dtype: dbdate

In [33]:

is_continuous = (country_all['date'].diff().dropna() == pd.Timedelta(days=1)).all()

is_continuous


True

### 3.4. W jaki sposób zapisywane są wartości liczbowe.

In [34]:
country_all.dtypes.value_counts()

float64    441
Int64      217
object      42
dbdate       1
Name: count, dtype: int64

Przykłady kolumn, które powiny być typem liczbowym a są obiektami czyli ciągami znaków.

In [35]:
# Wszystkie age_bins{1-9}
display(country_all[['population_largest_city', 'population_clustered', 'human_capital_index', 'area_rural_sq_km', 'area_urban_sq_km', 'life_expectancy']])

,population_largest_city,population_clustered,human_capital_index,area_rural_sq_km,area_urban_sq_km,life_expectancy
0,1775933,1775933,0.747,277840,30501,77.7
1,1775933,1775933,0.747,277840,30501,77.7
2,1775933,1775933,0.747,277840,30501,77.7
3,1775933,1775933,0.747,277840,30501,77.7
4,1775933,1775933,0.747,277840,30501,77.7
...,...,...,...,...,...,...
986,1775933,1775933,0.747,277840,30501,77.7
987,1775933,1775933,0.747,277840,30501,77.7
988,1775933,1775933,0.747,277840,30501,77.7
989,1775933,1775933,0.747,277840,30501,77.7


In [36]:
# Zapisane w typie coś / 1000
display(country_all.filter(like='_per_'))

,gdp_per_capita_usd,nurses_per_1000,physicians_per_1000,hospital_beds_per_1000
0,15222,6.8926,2.3788,NaN
1,15222,6.8926,2.3788,NaN
2,15222,6.8926,2.3788,NaN
3,15222,6.8926,2.3788,NaN
4,15222,6.8926,2.3788,NaN
...,...,...,...,...
986,15222,6.8926,2.3788,NaN
987,15222,6.8926,2.3788,NaN
988,15222,6.8926,2.3788,NaN
989,15222,6.8926,2.3788,NaN


###  3.5. Jaki przedział czasowy jest uwzględniony w danych. Jakie są przedziały czasowe dla przypadków nowych zachorowań, nowych śmierci oraz nowych zaszczepionych osób.

In [37]:
query_new_confirmed = ("""
SELECT date, SUM(new_confirmed)
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE aggregation_level=0
GROUP BY date
HAVING SUM(new_confirmed) IS NOT NULL AND SUM(new_confirmed) > 0
ORDER BY date ASC
""")

query_new_deceased = ("""
SELECT date, SUM(new_deceased)
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE aggregation_level=0
GROUP BY date
HAVING SUM(new_deceased) IS NOT NULL AND SUM(new_deceased) > 0
ORDER BY date ASC
""")

query_new_person_vaccinated = ("""
SELECT date, SUM(new_persons_vaccinated)
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE aggregation_level=0
GROUP BY date
HAVING SUM(new_persons_vaccinated) IS NOT NULL AND SUM(new_persons_vaccinated) > 0
ORDER BY date ASC
""")

new_confirmed = client.query(query_new_confirmed).to_dataframe()
new_deceased = client.query(query_new_deceased).to_dataframe()
new_person_vaccinated = client.query(query_new_person_vaccinated).to_dataframe()

In [38]:
print(f"Przedział czasowy danych jest od {country_all['date'].min()} do {country_all['date'].max()}.")
print(f"Przedział czasowy new_confirmed jest od {new_confirmed['date'].min()} do {new_confirmed['date'].max()}.")
print(f"Przedział czasowy new_deceased jest od {new_deceased['date'].min()} do {new_deceased['date'].max()}.")
print(f"Przedział czasowy new_person_vaccinated jest od {new_person_vaccinated['date'].min()} do {new_person_vaccinated['date'].max()}.")


Przedział czasowy danych jest od 2020-01-01 do 2022-09-17.
Przedział czasowy new_confirmed jest od 2020-01-01 do 2022-09-15.
Przedział czasowy new_deceased jest od 2020-01-02 do 2022-09-15.
Przedział czasowy new_person_vaccinated jest od 2020-12-07 do 2022-09-14.


In [39]:
# Dla krajów z osobna
query = """
SELECT country_name, 
       MIN(date) AS start_date, 
       MAX(date) AS end_date,
       'new_confirmed' AS metric
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE new_confirmed IS NOT NULL AND new_confirmed > 0 AND aggregation_level=0
GROUP BY country_name

UNION ALL

SELECT country_name, 
       MIN(date) AS start_date, 
       MAX(date) AS end_date,
       'new_deceased' AS metric
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE new_deceased IS NOT NULL AND new_deceased > 0 AND aggregation_level=0
GROUP BY country_name

UNION ALL

SELECT country_name, 
       MIN(date) AS start_date, 
       MAX(date) AS end_date,
       'new_persons_vaccinated' AS metric
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE new_persons_vaccinated IS NOT NULL AND new_persons_vaccinated > 0 AND aggregation_level=0
GROUP BY country_name
ORDER BY country_name, metric
"""

dates_info = client.query(query).to_dataframe()
dates_info

,country_name,start_date,end_date,metric
0,Afghanistan,2020-02-22,2022-09-13,new_confirmed
1,Afghanistan,2020-03-21,2022-09-12,new_deceased
2,Afghanistan,2021-02-28,2022-09-12,new_persons_vaccinated
3,Albania,2020-03-07,2022-09-13,new_confirmed
4,Albania,2020-03-09,2022-09-11,new_deceased
...,...,...,...,...
663,Zambia,2020-01-02,2022-09-10,new_deceased
664,Zambia,2021-04-15,2022-08-07,new_persons_vaccinated
665,Zimbabwe,2020-01-02,2022-09-12,new_confirmed
666,Zimbabwe,2020-01-02,2022-08-29,new_deceased


###  3.6. Sprawdź więcej informacji (co najmniej 5 różnych) o danych dotyczących COVID-19. W tym celu nie wykonuj żadnych dodatkowych obliczeń. 


1. Ile kolumn liczy zbiór danych

In [40]:
num_columns = country_all.shape[1]
print(f"Liczba kolumn: {num_columns}")

Liczba kolumn: 701


2. Które państwa podawały dane z podziałem na płeć

In [41]:
query = ("""
SELECT country_name AS `Country_name`, iso_3166_1_alpha_3 AS `3_letter_code`, SUM(new_confirmed_male) AS confirmed_male, SUM(new_confirmed_female) AS confirmed_female, SUM(new_deceased_male) AS deceased_male, SUM(new_deceased_female) as deceased_female, SUM(new_tested_male) AS tested_male, SUM(new_tested_female) AS tested_female, SUM(new_hospitalized_patients_male) AS hospitalized_male, SUM(new_hospitalized_patients_female) AS hospitalized_female, SUM(new_intensive_care_patients_male) AS cared_male, SUM(new_intensive_care_patients_female) AS cared_female, SUM(new_recovered_male) AS recovered_male, SUM(new_recovered_female) AS recovered_female
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE aggregation_level = 0 
GROUP BY country_name, iso_3166_1_alpha_3
HAVING SUM(new_confirmed_male) IS NOT NULL OR SUM(new_deceased_male) IS NOT NULL OR SUM(new_tested_male) IS NOT NULL OR SUM(new_hospitalized_patients_male) IS NOT NULL OR SUM(new_recovered_male) IS NOT NULL
ORDER BY country_name
""")

df_temp= client.query(query).to_dataframe()
df_temp

,Country_name,3_letter_code,confirmed_male,confirmed_female,deceased_male,deceased_female,tested_male,tested_female,hospitalized_male,hospitalized_female,cared_male,cared_female,recovered_male,recovered_female
0,Argentina,ARG,4418859,4780139,72801,53930,13178569,14928395,248464,222232,48422,33209,<NA>,<NA>
1,Brazil,BRA,8875904,8368673,311308,244105,26382353,26921275,996777,792931,330847,245941,0,0
2,Colombia,COL,2697358,3042821,93611,62159,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2542476,2898545
3,Czech Republic,CZE,1956993,2111418,23149,17783,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1925067,2082491
4,Estonia,EST,272596,326685,<NA>,<NA>,1595674,1890648,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
5,Finland,FIN,526714,587525,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
6,Germany,DEU,6092181,6356325,64006,56007,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,4370388,4550358
7,Hong Kong,HKG,7417,<NA>,127,<NA>,<NA>,<NA>,6708,<NA>,<NA>,<NA>,<NA>,<NA>
8,India,IND,92449,51846,30,13,<NA>,<NA>,3431,1712,<NA>,<NA>,<NA>,<NA>
9,Malaysia,MYS,1676866,1323134,20855,15444,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


3. Które państwa podawały dane z podziałem na kategorie wiekowe

In [42]:
query = ("""
SELECT country_name AS `Country_name`, iso_3166_1_alpha_3 AS `3_letter_code`, SUM(new_confirmed_age_0) AS confirmed_age_0, SUM(new_deceased_age_0) AS deceased_age_0, SUM(new_tested_age_0) AS tested_age_0, SUM(new_hospitalized_patients_age_0) as hospitalized_age_0, SUM(new_intensive_care_patients_age_0) AS cared_age_0, SUM(new_recovered_age_0) AS recovered_age_0 
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE aggregation_level = 0 
GROUP BY country_name, iso_3166_1_alpha_3
HAVING SUM(new_confirmed_age_0) IS NOT NULL OR SUM(new_deceased_age_0) IS NOT NULL OR SUM(new_tested_age_0) IS NOT NULL OR SUM(new_hospitalized_patients_age_0) IS NOT NULL OR SUM(new_recovered_age_0) IS NOT NULL
ORDER BY country_name
""")

df_temp= client.query(query).to_dataframe()
df_temp

,Country_name,3_letter_code,confirmed_age_0,deceased_age_0,tested_age_0,hospitalized_age_0,cared_age_0,recovered_age_0
0,Argentina,ARG,224240,192,1303477,41095,3269,<NA>
1,Brazil,BRA,891485,1214,2726462,21195,5273,0
2,Colombia,COL,198102,360,<NA>,<NA>,<NA>,191387
3,Czech Republic,CZE,321365,7,<NA>,<NA>,<NA>,320168
4,Estonia,EST,44622,<NA>,208798,<NA>,<NA>,<NA>
5,Finland,FIN,85684,<NA>,<NA>,<NA>,<NA>,<NA>
6,France,FRA,2789829,<NA>,23039639,<NA>,<NA>,<NA>
7,Germany,DEU,437029,23,<NA>,<NA>,<NA>,290555
8,Hong Kong,HKG,666,0,<NA>,578,<NA>,<NA>
9,India,IND,4273,1,<NA>,78,<NA>,<NA>


4. Które państwa podawały dane dotyczące szczepień z podziałem na rodzaj(firmę) szczepionki

In [43]:
query = ("""
SELECT country_name AS `Country_name`, iso_3166_1_alpha_3 AS `3_letter_code`, SUM(new_persons_fully_vaccinated_moderna) AS vaccinated_moderna, SUM(new_persons_fully_vaccinated_janssen) AS vaccinated_janssen, SUM(new_persons_fully_vaccinated_pfizer) AS vaccinated_pfizer, SUM(new_persons_vaccinated) AS total_vaccinated 
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE aggregation_level = 0 
GROUP BY country_name, iso_3166_1_alpha_3
HAVING SUM(new_persons_fully_vaccinated_moderna) IS NOT NULL OR SUM(new_persons_fully_vaccinated_janssen) IS NOT NULL OR SUM(new_persons_fully_vaccinated_pfizer) IS NOT NULL
ORDER BY country_name
""")

df_temp= client.query(query).to_dataframe()
df_temp

,Country_name,3_letter_code,vaccinated_moderna,vaccinated_janssen,vaccinated_pfizer,total_vaccinated
0,Malaysia,MYS,<NA>,<NA>,15153452,27880068
1,United States of America,USA,63469471,16886719,112344792,263883327


5. Jakie jest znaczenie wartośći liczbowych z kolumn dotyczących restrykcji (school_closing, workplace_closing, cancel_public_events, restrictions on gatherings)

In [44]:
query = ("""
SELECT DISTINCT date, iso_3166_1_alpha_3 AS `3_letter_code`, CAST(school_closing AS INT64) AS `school_closing`, CAST(workplace_closing  AS INT64) AS `workplace_closing`, CAST(cancel_public_events AS INT64) AS `cancel_public_events`, CAST(restrictions_on_gatherings AS INT64) AS `restrictions_on_gatherings`
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE aggregation_level = 0
GROUP BY iso_3166_1_alpha_3, date, school_closing, workplace_closing, cancel_public_events, restrictions_on_gatherings
ORDER BY date
""")
df_restrictions= client.query(query).to_dataframe()
df_restrictions

,date,3_letter_code,school_closing,workplace_closing,cancel_public_events,restrictions_on_gatherings
0,2020-01-01,PER,0,0,0,0
1,2020-01-01,SUR,0,0,0,0
2,2020-01-01,IRL,0,0,0,0
3,2020-01-01,GNQ,<NA>,<NA>,<NA>,<NA>
4,2020-01-01,BES,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...
243781,2022-09-17,MYS,<NA>,<NA>,<NA>,<NA>
243782,2022-09-17,OMN,<NA>,<NA>,<NA>,<NA>
243783,2022-09-17,MMR,<NA>,<NA>,<NA>,<NA>
243784,2022-09-17,CHL,<NA>,<NA>,<NA>,<NA>


<b>School closing:</b>
<br> 0 - brak zaleceń,
<br> 1 - zalecane zamknięcie lub wprowadzenie obostrzeń,
<br> 2 - nakaz zamknięcia niektórych typów szkół,
<br> 3 - nakaz zamknięcia wszystkich rodzai szkół. <br> <br>
<b>Workplace closing:</b>
<br> 0 - brak zaleceń,
<br> 1 - zalecane zamknięcie lub praca zdalna, albo wprowadzenie obostrzeń dla firm,
<br> 2 - nakaz zamknięcia niektórych sektorów lub obowiązkowa praca zdalna dla części pracowników,
<br> 3 - nakaz zamknięcia wszystkich miejsc pracy z wyjątkiem działalności niezbędnych (np. sklepy spożywcze. placówki medyczne). <br> <br>
<b>Cancel public events:</b>
<br> 0 - brak zaleceń,
<br> 1 - zalecane unikanie/odwoływanie,
<br> 2 - zakaz organizacji. <br> <br>
<b>Restrictions on gatherings:</b>
<br> 0 - brak ograniczeń,
<br> 1 - ograniczenia dla bardzo dużych zgromadzeń (powyżej 1000 osób),
<br> 2 - ograniczenia dla zgromadzeń od 101 do 1000 osób,
<br> 3 - ograniczenia dla zgromadzeń od 11 do 100 osób,
<br> 4 - ograniczenia dla zgromadzeń do 10 osób lub mniej.
<br>
via https://github.com/OxCGRT/covid-policy-dataset/blob/main/documentation_and_codebook.md <br>

# Część 4
Przeanalizuj poniższe przypadki. Zastanów się, jakie dane potrzebujesz do każdego z nich, a następnie zapisz je w osobnych, jak najprostszych obiektach DataFrame. Na tym etapie nie przetwarzaj ich. Zadbaj o czystość danych, m.in. usuń niepotrzebne puste wartości i duplikaty, zunifikuj sposób podawania informacji, zaproponuj sposób naprawy błędnych danych. Gotowe dane z obiektów DataFrame zapisz w osobnych plikach CSV.


### 4.1 Podstawowe dane o wszystkich krajach świata, które będą zrozumiałe dla człowieka oraz uniwersalne i potencjalnie przyszłościowe do dalszego przetwarzania.

In [45]:
query = ("""
SELECT DISTINCT country_name AS `Country_name`, iso_3166_1_alpha_3 AS `3_letter_code`, population, area_sq_km, population_density, population_urban, gdp_per_capita_usd
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE aggregation_level = 0
GROUP BY country_name, iso_3166_1_alpha_3, population, area_sq_km, population_density, population_urban, gdp_per_capita_usd
""")

country_data = client.query(query).to_dataframe()
country_data["GDP_2022"] = country_data["population"] * country_data["gdp_per_capita_usd"]
country_data

,Country_name,3_letter_code,population,area_sq_km,population_density,population_urban,gdp_per_capita_usd,GDP_2022
0,Finland,FIN,5517919,338450,18.100,4716888,50150,276723637850
1,Azerbaijan,AZE,10139175,86600,122.664,5616165,4793,48597065775
2,Egypt,EGY,102334403,1001450,102.802,42895824,3020,309049897060
3,Faroe Islands,FRO,48865,1396,35.004,20554,58622,2864564030
4,Greenland,GRL,56772,410450,0.138,48945,54470,3092370840
...,...,...,...,...,...,...,...,...
241,Swaziland,SWZ,1160164,17360,67.451,275333,3837,4451549268
242,Tokelau,TKL,1350,10,135.000,<NA>,<NA>,<NA>
243,Trinidad and Tobago,TTO,1399491,5130,272.805,741944,17276,24177606516
244,British Virgin Islands,VGB,30237,150,201.580,14449,<NA>,<NA>


### 4.2 statystyki dotyczące zachorowań na COVID-19 na całym świecie.

In [46]:
# Tutaj są wartości null w sumie. Można podmienić na 0 COALESCE(SUM(new_confirmed), 0) ale oznaczała by to, że nikt tam nie chorował a naprawdę nie ma danych.
query = ("""
SELECT date, iso_3166_1_alpha_3 AS `3_letter_code`, new_confirmed as `Total_confirmed_cases`,
FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
WHERE aggregation_level = 0
GROUP BY iso_3166_1_alpha_3, date, new_confirmed
ORDER BY date
""")

covid_statistic = client.query(query).to_dataframe()
covid_statistic.dropna(subset=['Total_confirmed_cases'], inplace=True)
covid_statistic

,date,3_letter_code,Total_confirmed_cases
0,2020-01-01,FIN,0
2,2020-01-01,PRI,0
4,2020-01-01,VGB,0
5,2020-01-01,MDA,0
6,2020-01-01,MKD,0
...,...,...,...
243194,2022-09-15,FRA,33263
243200,2022-09-15,THA,1125
243243,2022-09-15,IDN,2651
243260,2022-09-15,AUS,1796


### 4.3 Uwypuklenie problemu śmiertelności.

In [47]:
query = ("""
SELECT date,iso_3166_1_alpha_3 AS `3_letter_code`, new_deceased as `Total_new_deceased` 
FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
WHERE aggregation_level = 0
GROUP BY iso_3166_1_alpha_3, date, new_deceased
""")

covid_deceased = client.query(query).to_dataframe()
covid_deceased.dropna(subset=['Total_new_deceased'], inplace=True)
covid_deceased

,date,3_letter_code,Total_new_deceased
0,2021-05-17,ARG,537
1,2021-12-15,COL,76
2,2021-04-27,IND,3645
3,2021-08-07,PER,52
4,2022-01-11,PER,52
...,...,...,...
243781,2021-03-01,ARE,15
243782,2020-10-19,AUT,15
243783,2022-03-21,GTM,15
243784,2020-07-18,GBR,15


### 4.4 Trendy i zależności dotyczące szczepień na COVID-19.

In [48]:
query = ("""
SELECT date, iso_3166_1_alpha_3 AS `3_letter_code`,
new_persons_vaccinated as `Total_new_vaccinations`,
new_persons_fully_vaccinated as`New_persons_fully_vaccinated`
FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
WHERE aggregation_level = 0
GROUP BY iso_3166_1_alpha_3, date, new_persons_vaccinated, new_persons_fully_vaccinated
""")

covid_trend_analysis = client.query(query).to_dataframe()
covid_trend_analysis.dropna(subset=['Total_new_vaccinations', 'New_persons_fully_vaccinated'], how='all', inplace=True)
covid_trend_analysis

,date,3_letter_code,Total_new_vaccinations,New_persons_fully_vaccinated
0,2022-07-08,EST,35,35
1,2021-08-02,EST,3828,2902
2,2022-04-08,FRA,2063,2545
3,2021-10-24,IND,3592177,5038242
4,2021-12-19,NZL,785,3372
...,...,...,...,...
243781,2022-06-25,LIE,0,0
243782,2021-09-04,MHL,1,0
243783,2021-12-17,VNM,416075,0
243784,2021-11-02,VNM,435727,0


### 4.5 Liczba wykonanych testów

In [49]:
query = ("""
SELECT date, iso_3166_1_alpha_3 AS `3_letter_code`, CAST(investment_in_vaccines AS INT64) as `Investment_in_vaccines`, CAST(new_tested AS INT64) as `Total_tested`
FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
WHERE aggregation_level = 0
GROUP BY iso_3166_1_alpha_3, date, investment_in_vaccines, new_tested
ORDER BY date
""")

investment_in_vaccines = client.query(query).to_dataframe()
investment_in_vaccines.dropna(subset=['Total_tested'], how='all', inplace=True)

display(investment_in_vaccines)

,date,3_letter_code,Investment_in_vaccines,Total_tested
20,2020-01-01,MEX,0,72
53,2020-01-01,ARG,0,4
110,2020-01-01,BRA,0,1154
341,2020-01-02,ARG,0,42
364,2020-01-02,BRA,0,2337
...,...,...,...,...
243048,2022-09-15,AUT,<NA>,67392
243092,2022-09-15,AUS,<NA>,4367
243184,2022-09-15,MYS,<NA>,0
243212,2022-09-15,ITA,<NA>,146983


In [50]:
df_restrictions

,date,3_letter_code,school_closing,workplace_closing,cancel_public_events,restrictions_on_gatherings
0,2020-01-01,PER,0,0,0,0
1,2020-01-01,SUR,0,0,0,0
2,2020-01-01,IRL,0,0,0,0
3,2020-01-01,GNQ,<NA>,<NA>,<NA>,<NA>
4,2020-01-01,BES,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...
243781,2022-09-17,MYS,<NA>,<NA>,<NA>,<NA>
243782,2022-09-17,OMN,<NA>,<NA>,<NA>,<NA>
243783,2022-09-17,MMR,<NA>,<NA>,<NA>,<NA>
243784,2022-09-17,CHL,<NA>,<NA>,<NA>,<NA>


# Część 5

Połącz ze sobą wszystkie dane otrzymane w części 4. Nowy zbiór danych zapisz jako oddzielny obiekt DataFrame. Jeżeli uważasz, że należy stworzyć kilka takich obiektów, zrób to i zapisz swoje uzasadnienie. Pamiętaj o ciągłej konieczności zachowania czystości danych. Wynik tego zadania zapisz w pliku / plikach CSV.

In [52]:
df_data_from_4 = pd.merge(country_data, covid_statistic, on=['3_letter_code'])
df_data_from_4 = pd.merge(df_data_from_4, covid_deceased, on=['date', '3_letter_code'])
df_data_from_4 = pd.merge(df_data_from_4, covid_trend_analysis, on=['date', '3_letter_code'])
df_data_from_4 = pd.merge(df_data_from_4, investment_in_vaccines, on=['date', '3_letter_code'])
df_data_from_4 = pd.merge(df_data_from_4, df_restrictions, on =['date', '3_letter_code'])
df_data_from_4.to_csv("./wyniki/data_from_4.csv", index=False)
df_data_from_4

,Country_name,3_letter_code,population,area_sq_km,population_density,population_urban,gdp_per_capita_usd,GDP_2022,date,Total_confirmed_cases,Total_new_deceased,Total_new_vaccinations,New_persons_fully_vaccinated,Investment_in_vaccines,Total_tested,school_closing,workplace_closing,cancel_public_events,restrictions_on_gatherings
0,Finland,FIN,5517919,338450,18.100,4716888,50150,276723637850,2021-01-17,236,1,62050,<NA>,0,9367,2,2,2,4
1,Finland,FIN,5517919,338450,18.100,4716888,50150,276723637850,2021-01-24,246,5,53248,4957,0,11362,2,2,2,4
2,Finland,FIN,5517919,338450,18.100,4716888,50150,276723637850,2021-01-31,367,6,41790,14719,0,12904,2,2,2,4
3,Finland,FIN,5517919,338450,18.100,4716888,50150,276723637850,2021-02-07,354,5,22331,33340,0,14031,2,2,2,4
4,Finland,FIN,5517919,338450,18.100,4716888,50150,276723637850,2021-02-14,312,1,39386,16883,0,15047,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31435,Trinidad and Tobago,TTO,1399491,5130,272.805,741944,17276,24177606516,2022-06-12,217,4,98,49,0,531,0,0,0,0
31436,Trinidad and Tobago,TTO,1399491,5130,272.805,741944,17276,24177606516,2022-06-14,121,2,48,35,0,1809,0,0,0,0
31437,Trinidad and Tobago,TTO,1399491,5130,272.805,741944,17276,24177606516,2022-06-18,106,4,85,43,0,446,0,0,0,0
31438,Trinidad and Tobago,TTO,1399491,5130,272.805,741944,17276,24177606516,2022-06-20,101,5,70,70,0,512,0,0,0,0


# Część 6


### 6.1 Usunięcie powtarzających się kolumn, usuwamy z naszego zbioru, ponieważ zbiory znalezione potem okazują się być dokładniejsze.

In [53]:
df_data_from_4.drop('population', axis=1, inplace=True)
df_data_from_4.drop('area_sq_km', axis=1, inplace=True)
df_data_from_4.drop('population_density', axis=1, inplace=True)

df_data_from_4

,Country_name,3_letter_code,population_urban,gdp_per_capita_usd,GDP_2022,date,Total_confirmed_cases,Total_new_deceased,Total_new_vaccinations,New_persons_fully_vaccinated,Investment_in_vaccines,Total_tested,school_closing,workplace_closing,cancel_public_events,restrictions_on_gatherings
0,Finland,FIN,4716888,50150,276723637850,2021-01-17,236,1,62050,<NA>,0,9367,2,2,2,4
1,Finland,FIN,4716888,50150,276723637850,2021-01-24,246,5,53248,4957,0,11362,2,2,2,4
2,Finland,FIN,4716888,50150,276723637850,2021-01-31,367,6,41790,14719,0,12904,2,2,2,4
3,Finland,FIN,4716888,50150,276723637850,2021-02-07,354,5,22331,33340,0,14031,2,2,2,4
4,Finland,FIN,4716888,50150,276723637850,2021-02-14,312,1,39386,16883,0,15047,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31435,Trinidad and Tobago,TTO,741944,17276,24177606516,2022-06-12,217,4,98,49,0,531,0,0,0,0
31436,Trinidad and Tobago,TTO,741944,17276,24177606516,2022-06-14,121,2,48,35,0,1809,0,0,0,0
31437,Trinidad and Tobago,TTO,741944,17276,24177606516,2022-06-18,106,4,85,43,0,446,0,0,0,0
31438,Trinidad and Tobago,TTO,741944,17276,24177606516,2022-06-20,101,5,70,70,0,512,0,0,0,0


### 6.2 Podstawowe infromacje geograficzne i demograficzne o krajach

In [54]:
# Tutaj możemy usunąć dane z 4.1, ponieważ są one już w pliku .csv z 4.1

world_countries = pd.read_csv("world_countries.csv")

df_with_countries = pd.merge(df_data_from_4, world_countries, left_on=['3_letter_code'], right_on=['CCA3'], how="left")
df_with_countries.drop('Country/Territory', axis=1, inplace=True)
df_with_countries.drop('CCA3', axis=1, inplace=True)
df_with_countries["2021 Population"] = (df_with_countries["2020 Population"] + df_with_countries["2022 Population"]) / 2
df_with_countries

,Country_name,3_letter_code,population_urban,gdp_per_capita_usd,GDP_2022,date,Total_confirmed_cases,Total_new_deceased,Total_new_vaccinations,New_persons_fully_vaccinated,...,2010 Population,2000 Population,1990 Population,1980 Population,1970 Population,Area (km²),Density (per km²),Growth Rate,World Population Percentage,2021 Population
0,Finland,FIN,4716888,50150,276723637850,2021-01-17,236,1,62050,<NA>,...,5363271,5176209,4986545,4779418,4606621,338424,16.3722,1.0009,0.07,5535106.5
1,Finland,FIN,4716888,50150,276723637850,2021-01-24,246,5,53248,4957,...,5363271,5176209,4986545,4779418,4606621,338424,16.3722,1.0009,0.07,5535106.5
2,Finland,FIN,4716888,50150,276723637850,2021-01-31,367,6,41790,14719,...,5363271,5176209,4986545,4779418,4606621,338424,16.3722,1.0009,0.07,5535106.5
3,Finland,FIN,4716888,50150,276723637850,2021-02-07,354,5,22331,33340,...,5363271,5176209,4986545,4779418,4606621,338424,16.3722,1.0009,0.07,5535106.5
4,Finland,FIN,4716888,50150,276723637850,2021-02-14,312,1,39386,16883,...,5363271,5176209,4986545,4779418,4606621,338424,16.3722,1.0009,0.07,5535106.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31435,Trinidad and Tobago,TTO,741944,17276,24177606516,2022-06-12,217,4,98,49,...,1410296,1332203,1266518,1127852,988890,5130,298.4491,1.0035,0.02,1524595.5
31436,Trinidad and Tobago,TTO,741944,17276,24177606516,2022-06-14,121,2,48,35,...,1410296,1332203,1266518,1127852,988890,5130,298.4491,1.0035,0.02,1524595.5
31437,Trinidad and Tobago,TTO,741944,17276,24177606516,2022-06-18,106,4,85,43,...,1410296,1332203,1266518,1127852,988890,5130,298.4491,1.0035,0.02,1524595.5
31438,Trinidad and Tobago,TTO,741944,17276,24177606516,2022-06-20,101,5,70,70,...,1410296,1332203,1266518,1127852,988890,5130,298.4491,1.0035,0.02,1524595.5


In [55]:
df_gdp = pd.read_csv("gdp.csv")
df_gdp_pivot = df_gdp.pivot(index="Country Code", columns="Year", values="Value")
df_gdp_pivot.columns = [f"GDP_{int(year)}" for year in df_gdp_pivot.columns]
df_gdp_pivot.reset_index(inplace=True)
df_with_gdp = df_with_countries.merge(df_gdp_pivot, left_on="3_letter_code", right_on="Country Code", how="left")
df_with_gdp.drop(columns=["Country Code"], inplace=True)
display(df_with_gdp)

,Country_name,3_letter_code,population_urban,gdp_per_capita_usd,GDP_2022,date,Total_confirmed_cases,Total_new_deceased,Total_new_vaccinations,New_persons_fully_vaccinated,...,GDP_2007,GDP_2008,GDP_2009,GDP_2010,GDP_2011,GDP_2012,GDP_2013,GDP_2014,GDP_2015,GDP_2016
0,Finland,FIN,4716888,50150,276723637850,2021-01-17,236,1,62050,<NA>,...,2.553846e+11,2.837425e+11,2.514990e+11,2.477998e+11,2.736742e+11,2.567065e+11,2.699801e+11,2.726093e+11,2.324393e+11,2.385029e+11
1,Finland,FIN,4716888,50150,276723637850,2021-01-24,246,5,53248,4957,...,2.553846e+11,2.837425e+11,2.514990e+11,2.477998e+11,2.736742e+11,2.567065e+11,2.699801e+11,2.726093e+11,2.324393e+11,2.385029e+11
2,Finland,FIN,4716888,50150,276723637850,2021-01-31,367,6,41790,14719,...,2.553846e+11,2.837425e+11,2.514990e+11,2.477998e+11,2.736742e+11,2.567065e+11,2.699801e+11,2.726093e+11,2.324393e+11,2.385029e+11
3,Finland,FIN,4716888,50150,276723637850,2021-02-07,354,5,22331,33340,...,2.553846e+11,2.837425e+11,2.514990e+11,2.477998e+11,2.736742e+11,2.567065e+11,2.699801e+11,2.726093e+11,2.324393e+11,2.385029e+11
4,Finland,FIN,4716888,50150,276723637850,2021-02-14,312,1,39386,16883,...,2.553846e+11,2.837425e+11,2.514990e+11,2.477998e+11,2.736742e+11,2.567065e+11,2.699801e+11,2.726093e+11,2.324393e+11,2.385029e+11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31435,Trinidad and Tobago,TTO,741944,17276,24177606516,2022-06-12,217,4,98,49,...,2.164230e+10,2.787026e+10,1.917520e+10,2.215795e+10,2.543301e+10,2.569416e+10,2.643616e+10,2.617595e+10,2.355922e+10,2.189471e+10
31436,Trinidad and Tobago,TTO,741944,17276,24177606516,2022-06-14,121,2,48,35,...,2.164230e+10,2.787026e+10,1.917520e+10,2.215795e+10,2.543301e+10,2.569416e+10,2.643616e+10,2.617595e+10,2.355922e+10,2.189471e+10
31437,Trinidad and Tobago,TTO,741944,17276,24177606516,2022-06-18,106,4,85,43,...,2.164230e+10,2.787026e+10,1.917520e+10,2.215795e+10,2.543301e+10,2.569416e+10,2.643616e+10,2.617595e+10,2.355922e+10,2.189471e+10
31438,Trinidad and Tobago,TTO,741944,17276,24177606516,2022-06-20,101,5,70,70,...,2.164230e+10,2.787026e+10,1.917520e+10,2.215795e+10,2.543301e+10,2.569416e+10,2.643616e+10,2.617595e+10,2.355922e+10,2.189471e+10


# Część 7

### 7.1 Dane związane z inflacją

In [56]:
# https://www.worldbank.org/en/research/brief/inflation-database

# Z wszystkich strony arkusza wybraliśmy ogólny przyrost inflacji w danych latach
df_inflation = pd.read_excel("load_data\\inflation_all.xlsx", sheet_name="hcpi_a")

df_inflation.drop(columns=["IMF Country Code", "Indicator Type", "Series Name", "Note", "Country"], inplace=True)
df_inflation = df_inflation.melt(id_vars=["Country Code"], var_name="Year", value_name="Inflation")
df_inflation.dropna(subset=["Inflation"], inplace=True)
df_inflation = df_inflation.pivot(index="Country Code", columns="Year", values="Inflation")
df_inflation.columns = [f"Inflation_{int(year)}" for year in df_inflation.columns]
df_inflation.reset_index(inplace=True)
df_with_inflation = df_with_gdp.merge(df_inflation, left_on="3_letter_code", right_on="Country Code", how="left")
df_with_inflation.drop(columns=["Country Code"], inplace=True)
display(df_with_inflation)

,Country_name,3_letter_code,population_urban,gdp_per_capita_usd,GDP_2022,date,Total_confirmed_cases,Total_new_deceased,Total_new_vaccinations,New_persons_fully_vaccinated,...,Inflation_2014,Inflation_2015,Inflation_2016,Inflation_2017,Inflation_2018,Inflation_2019,Inflation_2020,Inflation_2021,Inflation_2022,Inflation_2023
0,Finland,FIN,4716888,50150,276723637850,2021-01-17,236,1,62050,<NA>,...,1.041196,-0.207928,0.356684,0.839,1.08,1.137,0.290555,2.194574,7.123507,6.250643
1,Finland,FIN,4716888,50150,276723637850,2021-01-24,246,5,53248,4957,...,1.041196,-0.207928,0.356684,0.839,1.08,1.137,0.290555,2.194574,7.123507,6.250643
2,Finland,FIN,4716888,50150,276723637850,2021-01-31,367,6,41790,14719,...,1.041196,-0.207928,0.356684,0.839,1.08,1.137,0.290555,2.194574,7.123507,6.250643
3,Finland,FIN,4716888,50150,276723637850,2021-02-07,354,5,22331,33340,...,1.041196,-0.207928,0.356684,0.839,1.08,1.137,0.290555,2.194574,7.123507,6.250643
4,Finland,FIN,4716888,50150,276723637850,2021-02-14,312,1,39386,16883,...,1.041196,-0.207928,0.356684,0.839,1.08,1.137,0.290555,2.194574,7.123507,6.250643
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31435,Trinidad and Tobago,TTO,741944,17276,24177606516,2022-06-12,217,4,98,49,...,5.689053,4.659380,3.094139,1.884,1.03,1.001,0.598986,2.061069,5.543084,5.917354
31436,Trinidad and Tobago,TTO,741944,17276,24177606516,2022-06-14,121,2,48,35,...,5.689053,4.659380,3.094139,1.884,1.03,1.001,0.598986,2.061069,5.543084,5.917354
31437,Trinidad and Tobago,TTO,741944,17276,24177606516,2022-06-18,106,4,85,43,...,5.689053,4.659380,3.094139,1.884,1.03,1.001,0.598986,2.061069,5.543084,5.917354
31438,Trinidad and Tobago,TTO,741944,17276,24177606516,2022-06-20,101,5,70,70,...,5.689053,4.659380,3.094139,1.884,1.03,1.001,0.598986,2.061069,5.543084,5.917354


### Dane o bezrobociu

In [57]:
#  https://data.worldbank.org/indicator/SL.UEM.TOTL.ZS?view=map&year=1991

df_unemployment = pd.read_csv("load_data\\total_unemployment.csv")
df_unemployment.drop(columns=["Country Name", "Indicator Name", "Indicator Code", "Unnamed: 68"], inplace=True, errors="ignore")
df_unemployment = df_unemployment.melt(id_vars=["Country Code"], var_name="Year", value_name="Unemployment")
df_unemployment.dropna(subset=["Unemployment"], inplace=True)
df_unemployment = df_unemployment.pivot(index="Country Code", columns="Year", values="Unemployment")
df_unemployment.columns = [f"Unemployment_{int(year)}" for year in df_unemployment.columns]
df_unemployment.reset_index(inplace=True)
df_unemployment = df_with_inflation.merge(df_unemployment, left_on="3_letter_code", right_on="Country Code", how="left")
df_unemployment.drop(columns=["Country Code"], inplace=True)
display(df_unemployment)

,Country_name,3_letter_code,population_urban,gdp_per_capita_usd,GDP_2022,date,Total_confirmed_cases,Total_new_deceased,Total_new_vaccinations,New_persons_fully_vaccinated,...,Unemployment_2014,Unemployment_2015,Unemployment_2016,Unemployment_2017,Unemployment_2018,Unemployment_2019,Unemployment_2020,Unemployment_2021,Unemployment_2022,Unemployment_2023
0,Finland,FIN,4716888,50150,276723637850,2021-01-17,236,1,62050,<NA>,...,8.663,9.376,8.818,8.640,7.361,6.695,7.759,7.617,6.719,7.151
1,Finland,FIN,4716888,50150,276723637850,2021-01-24,246,5,53248,4957,...,8.663,9.376,8.818,8.640,7.361,6.695,7.759,7.617,6.719,7.151
2,Finland,FIN,4716888,50150,276723637850,2021-01-31,367,6,41790,14719,...,8.663,9.376,8.818,8.640,7.361,6.695,7.759,7.617,6.719,7.151
3,Finland,FIN,4716888,50150,276723637850,2021-02-07,354,5,22331,33340,...,8.663,9.376,8.818,8.640,7.361,6.695,7.759,7.617,6.719,7.151
4,Finland,FIN,4716888,50150,276723637850,2021-02-14,312,1,39386,16883,...,8.663,9.376,8.818,8.640,7.361,6.695,7.759,7.617,6.719,7.151
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31435,Trinidad and Tobago,TTO,741944,17276,24177606516,2022-06-12,217,4,98,49,...,2.477,2.428,3.207,3.394,3.498,3.523,4.206,4.446,4.383,4.210
31436,Trinidad and Tobago,TTO,741944,17276,24177606516,2022-06-14,121,2,48,35,...,2.477,2.428,3.207,3.394,3.498,3.523,4.206,4.446,4.383,4.210
31437,Trinidad and Tobago,TTO,741944,17276,24177606516,2022-06-18,106,4,85,43,...,2.477,2.428,3.207,3.394,3.498,3.523,4.206,4.446,4.383,4.210
31438,Trinidad and Tobago,TTO,741944,17276,24177606516,2022-06-20,101,5,70,70,...,2.477,2.428,3.207,3.394,3.498,3.523,4.206,4.446,4.383,4.210


### Dane o przestępstwach w latach

In [58]:
df_crime = pd.read_csv("load_data\\crime_rate_per_100k.csv")
df_crime = df_crime.iloc[:217]
df_crime.drop(columns=["Series Name", "Series Code", "Country Name"], inplace=True, errors="ignore")
df_crime.columns = df_crime.columns.str.replace(r"\s*\[YR\d+\]", "", regex=True)
df_crime = df_crime.melt(id_vars=["Country Code"], var_name="Year", value_name="Crime Rate")
df_crime["Year"] = df_crime["Year"].astype(int)
df_crime["Crime Rate"] = pd.to_numeric(df_crime["Crime Rate"], errors="coerce")
df_crime.dropna(subset=["Crime Rate"], inplace=True)
df_crime = df_crime.pivot(index="Country Code", columns="Year", values="Crime Rate")
df_crime.columns = [f"Murders_Rate_{int(year)}" for year in df_crime.columns]
df_crime.reset_index(inplace=True)
df_final = df_unemployment.merge(df_crime, left_on="3_letter_code", right_on="Country Code", how="left")
df_final.drop(columns=["Country Code"], inplace=True)
df_final.to_csv("./wyniki/final_data.csv",  index=False)
display(df_final)

,Country_name,3_letter_code,population_urban,gdp_per_capita_usd,GDP_2022,date,Total_confirmed_cases,Total_new_deceased,Total_new_vaccinations,New_persons_fully_vaccinated,...,Murders_Rate_2012,Murders_Rate_2013,Murders_Rate_2014,Murders_Rate_2015,Murders_Rate_2016,Murders_Rate_2017,Murders_Rate_2018,Murders_Rate_2019,Murders_Rate_2020,Murders_Rate_2021
0,Finland,FIN,4716888,50150,276723637850,2021-01-17,236,1,62050,<NA>,...,1.643929,1.636369,1.611297,1.496498,1.346625,1.216382,1.559253,1.466983,1.645728,NaN
1,Finland,FIN,4716888,50150,276723637850,2021-01-24,246,5,53248,4957,...,1.643929,1.636369,1.611297,1.496498,1.346625,1.216382,1.559253,1.466983,1.645728,NaN
2,Finland,FIN,4716888,50150,276723637850,2021-01-31,367,6,41790,14719,...,1.643929,1.636369,1.611297,1.496498,1.346625,1.216382,1.559253,1.466983,1.645728,NaN
3,Finland,FIN,4716888,50150,276723637850,2021-02-07,354,5,22331,33340,...,1.643929,1.636369,1.611297,1.496498,1.346625,1.216382,1.559253,1.466983,1.645728,NaN
4,Finland,FIN,4716888,50150,276723637850,2021-02-14,312,1,39386,16883,...,1.643929,1.636369,1.611297,1.496498,1.346625,1.216382,1.559253,1.466983,1.645728,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31435,Trinidad and Tobago,TTO,741944,17276,24177606516,2022-06-12,217,4,98,49,...,26.566432,28.249578,27.918308,28.763646,31.510972,33.477455,34.358803,35.461576,26.282040,29.364283
31436,Trinidad and Tobago,TTO,741944,17276,24177606516,2022-06-14,121,2,48,35,...,26.566432,28.249578,27.918308,28.763646,31.510972,33.477455,34.358803,35.461576,26.282040,29.364283
31437,Trinidad and Tobago,TTO,741944,17276,24177606516,2022-06-18,106,4,85,43,...,26.566432,28.249578,27.918308,28.763646,31.510972,33.477455,34.358803,35.461576,26.282040,29.364283
31438,Trinidad and Tobago,TTO,741944,17276,24177606516,2022-06-20,101,5,70,70,...,26.566432,28.249578,27.918308,28.763646,31.510972,33.477455,34.358803,35.461576,26.282040,29.364283
